# Mastercurve Model Comparison

Template to compare multiple models on mastercurve data (polystyrene).

**Data:** examples/data/pyRheo/polystyrene/master_curve_ps_oscillation_data.csv

In [1]:
# Google Colab compatibility - uncomment if running in Colab
# !pip install -q rheojax
# from google.colab import drive
# drive.mount('/content/drive')


## Setup and Imports

In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
%matplotlib inline

import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fractional_maxwell_model import FractionalMaxwellModel
from rheojax.models.generalized_maxwell import GeneralizedMaxwell
from rheojax.pipeline.base import Pipeline
from rheojax.transforms.mastercurve import Mastercurve

jax, jnp = safe_import_jax()
verify_float64()
np.set_printoptions(precision=4, suppress=True)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

def r2_complex(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum(np.abs(y_true - y_pred) ** 2)
    ss_tot = np.sum(np.abs(y_true - np.mean(y_true)) ** 2)
    return float(1 - ss_res / ss_tot)


INFO:2025-12-06 10:20:53,022:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Loading rheojax version 0.4.0


/Users/b80985/Projects/rheojax/.venv/lib/python3.13/site-packages/piblin_jax/__init__.py:28: UserWarning: GPU acceleration is only available on Linux with CUDA 12+. Using JAX in CPU mode.
  from . import backend, data, dataio, transform


## Load mastercurve (polystyrene)

In [3]:
DATA_DIR = Path.cwd().parent / 'data' / 'pyRheo' / 'polystyrene'
df = pd.read_csv(DATA_DIR / 'master_curve_ps_oscillation_data.csv')
omega = df['Angular Frequency'].to_numpy()
Gp = df['Storage Modulus'].to_numpy()
Gpp = df['Loss Modulus'].to_numpy()
G_star = Gp + 1j*Gpp


## Compare candidate models

In [4]:
candidates = [
    ('GM-3', GeneralizedMaxwell(n_modes=3, modulus_type='tensile')),
    ('GM-5', GeneralizedMaxwell(n_modes=5, modulus_type='tensile')),
    ('GM-7', GeneralizedMaxwell(n_modes=7, modulus_type='tensile')),
    ('FracMax', FractionalMaxwellModel()),
]
rows = []
for name, model in candidates:
    try:
        model.fit(omega, G_star, test_mode='oscillation', use_log_residuals=True)
        pred = model.predict(omega)
        if pred.ndim == 2:
            pred = pred[:,0] + 1j*pred[:,1]
        r2 = r2_complex(G_star, pred)
        rows.append({'model': name, 'r2': r2})
    except Exception as exc:
        rows.append({'model': name, 'r2': np.nan, 'error': str(exc)})

pd.DataFrame(rows)


Starting least squares optimization | {'method': 'trf', 'n_params': 7, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 1.317097s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=64 | final_cost=3.547677e+11 | time=1.317s | final_gradient_norm=248598245.35240754


Starting least squares optimization | {'method': 'trf', 'n_params': 7, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.384700s


...

Element minimization: early termination at n_modes=6 (R²=0.999233 < threshold=0.999368)


Starting least squares optimization | {'method': 'trf', 'n_params': 4, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 1.079096s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=30 | final_cost=7.099389e+00 | time=1.079s | final_gradient_norm=0.0065040219009454515


,model,r2
0,GM-3,0.993697
1,GM-5,0.982704
2,GM-7,0.999579
3,FracMax,-43733.637158


## Visualize top 2

In [5]:
top2 = sorted([r for r in rows if np.isfinite(r['r2'])], key=lambda x: x['r2'], reverse=True)[:2]
freq_hz = omega/(2*np.pi)
fig, axes = plt.subplots(1, len(top2), figsize=(14,6), sharey=True)
for ax, rec in zip(axes, top2):
    model = dict(candidates)[rec['model']]
    model.fit(omega, G_star, test_mode='oscillation', use_log_residuals=True)
    pred = model.predict(omega)
    if pred.ndim == 2:
        pred = pred[:,0] + 1j*pred[:,1]
    ax.loglog(freq_hz, np.real(G_star), 'o', label="G' data", alpha=0.5)
    ax.loglog(freq_hz, np.imag(G_star), 's', label='G" data', alpha=0.5)
    ax.loglog(freq_hz, np.real(pred), '-', label=f"G' {rec['model']}")
    ax.loglog(freq_hz, np.imag(pred), '--', label=f"Gpp {rec['model']}")
    ax.set_title(f"{rec['model']} (R^2={rec['r2']:.3f})")
    ax.grid(True, which='both', ls='--', alpha=0.4)
for ax in axes:
    ax.set_xlabel('Frequency (Hz)')
axes[0].set_ylabel('Modulus (Pa)')
axes[0].legend()
plt.show()


Starting least squares optimization | {'method': 'trf', 'n_params': 15, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.389321s


Convergence: reason=`xtol` termination condition is satisfied. | iterations=31 | final_cost=2.370154e+10 | time=0.389s | final_gradient_norm=2.5480554071252746e+20


Starting least squares optimization | {'method': 'trf', 'n_params': 15, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.388178s


...

Starting least squares optimization | {'method': 'trf', 'n_params': 5, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.208910s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=19 | final_cost=9.735034e+11 | time=0.209s | final_gradient_norm=1.5340190487145265e+19


Element minimization: early termination at n_modes=2 (R²=0.982707 < threshold=0.990547)


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_31334/2840107249.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
